In [1]:
import csv
from biomart import BiomartServer
#import time
import pandas as pd

In [2]:
# DICTIONARY
dictionary = {}

def gene_dictionary(index,
                    gene_name,
                    cluster,
                    gene_list_id,
                    ensembl_gene_id,
                    ensembl_transcript_id,
                    refseq_mrna,
                    hgnc_symbol,
                    alias,
                    direction):
    
    dictionary = {
        'Index' : index,
        'Gene_name' : gene_name,
        'Gene_list_id' : gene_list_id,
        'Ensembl_gene_id' : ensembl_gene_id,
        'Ensembl_transcript_id' : ensembl_transcript_id,
        'RefSeq_mRNA_id' : refseq_mrna,
        'HGNC_symbol' : hgnc_symbol,
        'UP' : cluster,
        'DOWN' : cluster,
        'Alias' : alias,
        'Info' : direction
    }
    
    return dictionary



# REGULATION
def regulation(gene_name):
    if gene_name in downTab:
        return 'Down'
    elif gene_name in upTab:
        return 'Up'
    else:
        return 'Unknown'
    

# RESPONSE  
def biomartParameters(mgi_symbol):

    attributes = ['ensembl_gene_id',
                  'ensembl_transcript_id',
                  'refseq_mrna']

    filters = {'mgi_symbol':[mgi_symbol]}                     # gene_name = mgi_symbol
    response = dataset.search({'attributes':attributes,'filters':filters})
    

# response_convertion
    lines = response.text.split("\n")
    lines = list(filter(lambda l: l != "",lines))
    values = []
    for l in lines:
        temp = []
        raw_values = l.split("\t")
        values.append(raw_values)
        
# if values not return ????

    return values  

def biomartHumanOrthologs(mgi_symbol):
    attributes = ['hsapiens_homolog_associated_gene_name']
    filters = {'mgi_symbol':[mgi_symbol]}
    response = dataset.search({'attributes':attributes,'filters':filters})
    
# response_convertion
    lines = response.text.split("\n")
    lines = list(filter(lambda l: l != "",lines))
    values = []
    for l in lines:
        temp = []
        raw_values = l.split("\t")
        values.append(raw_values)
        
# if values not return ????

    return values

In [3]:
# clusters_file
f = open('./Clusters_-_Symbols.tsv')
tsv_file = csv.reader(f, delimiter="\t")

# biomart_server
server = BiomartServer('http://www.ensembl.org/biomart')
dataset = server.datasets['mmusculus_gene_ensembl']


# separation_of_headers  
headers = next(tsv_file)               
rest2 = list(tsv_file)

# separation_of_regulation
downTab = [row[16] for row in rest2]
upTab = [row[17] for row in rest2]

# separation_of_rest
[row.pop(17) for row in rest2]   
[row.pop(16) for row in rest2]

rest = []
for row in rest2:
    for i in range(len(row)):
        if row[i] not in rest:
            rest.append(row[i])
            
#print(rest2)
#print(rest)

In [4]:
# CREATING FINAL LIST
ls_geneDictionaries = []
ls_biomartParameters = []
ls_biomartHumanOrthologs = []
ls_notResponse = []


for i in range(len(rest)):
    gene_name = rest[i]
#   print(gene_name)
    if gene_name != '':

        # variables

        geneSource = 'marpiech_tissues_dex'
        cluster = headers[i % 16]
        gene_list_id = geneSource + '_' + cluster
        index = i + 1
        direction = regulation(gene_name)          # => info
        alias = ''


        ensembl_gene_id_temp = []
        ensembl_transcript_id_temp = []
        refseq_mrna_temp = []

        ls_biomartParameters = biomartParameters(gene_name)
        if not ls_biomartParameters:
            ls_notResponse.append(gene_name)
#            print(gene_name)


        for ls in ls_biomartParameters:
            for j in range(3):
                if len(ls) < (j+1):
                    ls.append('')
            ensembl_gene_id_temp.append(ls[0])
            ensembl_transcript_id_temp.append(ls[1])
            refseq_mrna_temp.append(ls[2])  

        ensembl_gene_id = '|'.join(list(set(filter(None, ensembl_gene_id_temp))))
        ensembl_transcript_id = '|'.join(list(set(filter(None, ensembl_transcript_id_temp))))
        refseq_mrna = '|'.join(list(set(filter(None, refseq_mrna_temp))))
                               

        ls_biomartHumanOrthologs = biomartHumanOrthologs(gene_name)
        if not ls_biomartHumanOrthologs:
            hgnc_symbol = ''
        else:
            hgnc_symbol = ls_biomartHumanOrthologs[0][0]
        


        temp_gene_dictionary = gene_dictionary(index,
                                               gene_name,
                                               cluster,
                                               gene_list_id,
                                               ensembl_gene_id,
                                               ensembl_transcript_id, 
                                               refseq_mrna,
                                               hgnc_symbol,
                                               alias,
                                               direction)

        ls_geneDictionaries.append(temp_gene_dictionary)
        print(temp_gene_dictionary)

data = ls_geneDictionaries

df = pd.DataFrame.from_dict(data)



"""
for gene in ls_geneDictionaries:
print(gene)
"""

{'Index': 1, 'Gene_name': 'Mef2c', 'Gene_list_id': 'marpiech_tissues_dex_1', 'Ensembl_gene_id': 'ENSMUSG00000005583', 'Ensembl_transcript_id': 'ENSMUST00000198916|ENSMUST00000197022|ENSMUST00000197722|ENSMUST00000199262|ENSMUST00000198360|ENSMUST00000199167|ENSMUST00000198217|ENSMUST00000199210|ENSMUST00000197146|ENSMUST00000200123|ENSMUST00000197938|ENSMUST00000196493|ENSMUST00000200394|ENSMUST00000199105|ENSMUST00000163888|ENSMUST00000196207|ENSMUST00000195904|ENSMUST00000198199|ENSMUST00000195984|ENSMUST00000199450|ENSMUST00000199432|ENSMUST00000196832|ENSMUST00000200138|ENSMUST00000198069|ENSMUST00000199019|ENSMUST00000185052|ENSMUST00000005722|ENSMUST00000197681|ENSMUST00000198064|ENSMUST00000197145|ENSMUST00000196730', 'RefSeq_mRNA_id': 'NM_001347581|NM_001347580|NM_001347576|NM_001347566|NM_001347574|NM_001347573|NM_001347568|NM_001347575|NM_001347579|NM_025282|NM_001347571|NM_001347564|NM_001347572|NM_001347578|NM_001347569|NM_001347567|NM_001170537|NM_001347577', 'HGNC_symbol'

{'Index': 25, 'Gene_name': '9530053J19Rik', 'Gene_list_id': 'marpiech_tissues_dex_9', 'Ensembl_gene_id': '', 'Ensembl_transcript_id': '', 'RefSeq_mRNA_id': '', 'HGNC_symbol': '', 'UP': '9', 'DOWN': '9', 'Alias': '', 'Info': 'Up'}
{'Index': 26, 'Gene_name': 'Bmp4', 'Gene_list_id': 'marpiech_tissues_dex_10', 'Ensembl_gene_id': 'ENSMUSG00000021835', 'Ensembl_transcript_id': 'ENSMUST00000135408|ENSMUST00000074077|ENSMUST00000100676|ENSMUST00000226759', 'RefSeq_mRNA_id': 'NM_007554|NM_001316360', 'HGNC_symbol': 'BMP4', 'UP': '10', 'DOWN': '10', 'Alias': '', 'Info': 'Up'}
{'Index': 27, 'Gene_name': 'Tom1l2', 'Gene_list_id': 'marpiech_tissues_dex_11', 'Ensembl_gene_id': 'ENSMUSG00000000538', 'Ensembl_transcript_id': 'ENSMUST00000064019|ENSMUST00000143124|ENSMUST00000095254|ENSMUST00000151284|ENSMUST00000093046|ENSMUST00000133420|ENSMUST00000102682|ENSMUST00000153920|ENSMUST00000102683|ENSMUST00000093048|ENSMUST00000142225', 'RefSeq_mRNA_id': 'NM_001039092|NM_153080|NM_001403895|NM_001403896|N

KeyboardInterrupt: 

In [27]:
import openpyxl 


wb = openpyxl.load_workbook(r'C:\Users\joasi\geneBase\1_pub\Wyniki\response_1_pub_200323.xlsx')
ws = wb.active

mgi_id = [ws.cell(row=i,column=2).value for i in range(2,ws.max_row+1)]


gene_name_2 = [ws.cell(row=i,column=1).value for i in range(2,ws.max_row+1)]


print(mgi_id)




['MGI:95292', 'MGI:95292', 'MGI:95537', 'MGI:106023', 'MGI:107565', 'MGI:109382', 'MGI:88613', 'MGI:1277133', 'MGI:1314653', 'MGI:1858441', 'MGI:1860303', 'MGI:1914701', 'MGI:1915368', 'MGI:1915982', 'MGI:1916318', 'MGI:1918730', 'MGI:1921298', 'MGI:1921765', 'MGI:1922263', 'MGI:1924234', 'MGI:1924802', 'MGI:1924879', 'MGI:1924967', 'MGI:1933383', 'MGI:2441671', 'MGI:2442377', 'MGI:2442733', 'MGI:2442910', 'MGI:2443629', 'MGI:2444394', 'MGI:2670959', 'MGI:2685484', 'MGI:3643534', 'MGI:3645360', 'MGI:3646928', 'MGI:3648717']


In [ ]:
# SECOND RESPONSE
server = BiomartServer('http://www.ensembl.org/biomart')
dataset = server.datasets['mmusculus_gene_ensembl']

def biomartParameters_mgi(mgi_id):

    attributes = ['ensembl_gene_id',
                  'ensembl_transcript_id',
                  'refseq_mrna']
    print(mgi_id)
    filters = {'mgi_id':[mgi_id]}                     # gene_name = mgi_symbol
    response = dataset.search({'attributes':attributes,'filters':filters})
    

# response_convertion
    lines = response.text.split("\n")
    lines = list(filter(lambda l: l != "",lines))
    values = []
    for l in lines:
        temp = []
        raw_values = l.split("\t")
        values.append(raw_values)
        
# if values not return ????

    return values  

def biomartHumanOrthologs_mgi(mgi_id):
    attributes = ['hsapiens_homolog_associated_gene_name']
    
    filters = {'mgi_id':[mgi_id]}
    response = dataset.search({'attributes':attributes,'filters':filters})
    
# response_convertion
    lines = response.text.split("\n")
    lines = list(filter(lambda l: l != "",lines))
    values = []
    for l in lines:
        temp = []
        raw_values = l.split("\t")
        values.append(raw_values)
        
# if values not return ????

    return values

ls_notResponse_after = []
ls_biomartParameters = []
ls_biomartHumanOrthologs = []

for i in range(len(mgi_id)):
    mgi_ID = mgi_id[i]
    gene_name = gene_name_2[i]
    print(mgi_ID)
    if gene_name != '':

        # variables
        alias = ''


        ensembl_gene_id_temp = []
        ensembl_transcript_id_temp = []
        refseq_mrna_temp = []

        ls_biomartParameters = biomartParameters_mgi(mgi_ID)
        
        print(ls_biomartParameters)
        
        if not ls_biomartParameters:
            ls_notResponse_after.append(mgi_ID)
#            print(gene_name)


        for ls in ls_biomartParameters:
            for j in range(3):
                if len(ls) < (j+1):
                    ls.append('')
            ensembl_gene_id_temp.append(ls[0])
            ensembl_transcript_id_temp.append(ls[1])
            refseq_mrna_temp.append(ls[2])  

        ensembl_gene_id = '|'.join(list(set(filter(None, ensembl_gene_id_temp))))
        ensembl_transcript_id = '|'.join(list(set(filter(None, ensembl_transcript_id_temp))))
        refseq_mrna = '|'.join(list(set(filter(None, refseq_mrna_temp))))
                               

        ls_biomartHumanOrthologs = biomartHumanOrthologs_mgi(mgi_ID)
        if not ls_biomartHumanOrthologs:
            hgnc_symbol = ''
        else:
            hgnc_symbol = ls_biomartHumanOrthologs[0][0]
        print(ensembl_gene_id, ensembl_transcript_id, refseq_mrna, hgnc_symbol)
        for k in range(len(ls_geneDictionaries)):
            if ls_geneDictionaries[k]['Gene_name'] == gene_name:
                ls_geneDictionaries[k]['Ensembl_gene_id'] = ensembl_gene_id
                ls_geneDictionaries[k]['Ensembl_transcript_id'] = ensembl_transcript_id
                ls_geneDictionaries[k]['RefSeq_mRNA_id'] = refseq_mrna
                ls_geneDictionaries[k]['HGNC_symbol'] = hgnc_symbol
                

                
data = ls_geneDictionaries
df = pd.DataFrame.from_dict(data)




MGI:95292
[]
   
MGI:95292
[]
   
MGI:95537
[]
   
MGI:106023
[]
   
MGI:107565
[]
   
MGI:109382
[]
   
MGI:88613
[]
   
MGI:1277133
[]
   
MGI:1314653
[]
   
MGI:1858441
[]
   
MGI:1860303


In [ ]:
print()

In [12]:
# WYNIK_DATAFRAME
print(display(df))

,Index,Gene_name,Gene_list_id,Ensembl_gene_id,Ensembl_transcript_id,RefSeq_mRNA_id,HGNC_symbol,UP,DOWN,Alias,Info
0,1,Mef2c,marpiech_tissues_dex_1,ENSMUSG00000005583,ENSMUST00000199262|ENSMUST00000199019|ENSMUST0...,NM_001347581|NM_001347564|NM_001347568|NM_0013...,MEF2C,1,1,,Down
1,2,Cxcl1,marpiech_tissues_dex_2,ENSMUSG00000029380,ENSMUST00000201245|ENSMUST00000031327,NM_008176,,2,2,,Down
2,3,Id1,marpiech_tissues_dex_3,ENSMUSG00000042745,ENSMUST00000038368|ENSMUST00000109824,NM_001369018|NM_010495|NM_001355113,ID1,3,3,,Down
3,4,Aplnr,marpiech_tissues_dex_4,ENSMUSG00000044338,ENSMUST00000057019|ENSMUST00000184728,NM_011784,APLNR,4,4,,Down
4,5,Phlda2,marpiech_tissues_dex_5,ENSMUSG00000010760,ENSMUST00000207561|ENSMUST00000207425|ENSMUST0...,NM_009434,PHLDA2,5,5,,Down
...,...,...,...,...,...,...,...,...,...,...,...
474,476,Polr3e,marpiech_tissues_dex_12,ENSMUSG00000030880,ENSMUST00000209014|ENSMUST00000207481|ENSMUST0...,NM_001164096|NM_025298|NM_001379288,POLR3E,12,12,,Up
475,477,Arhgef28,marpiech_tissues_dex_13,ENSMUSG00000021662,ENSMUST00000109426|ENSMUST00000238746|ENSMUST0...,NM_001408685|NM_001408684|NM_012026|NM_001408686,ARHGEF28,13,13,,Up
476,478,Fgfbp1,marpiech_tissues_dex_14,ENSMUSG00000048373,ENSMUST00000199481|ENSMUST00000061299|ENSMUST0...,NM_008009|NM_001271616,FGFBP1,14,14,,Up
477,479,Uap1,marpiech_tissues_dex_15,ENSMUSG00000026670,ENSMUST00000191797|ENSMUST00000191690|ENSMUST0...,NM_133806|NM_001305046|NM_001305045,UAP1,15,15,,Up


None


In [13]:
# WYNIK_Excell
df.to_excel(r'C:\Users\joasi\geneBase\1_pub\Wyniki\geneBase_probe_2003233.xlsx', index=False)

In [10]:
# WYNIK_TSV
df.to_csv(r'C:\Users\joasi\geneBase\1_pub\Wyniki\geneBase_170323.tsv', sep="\t")

In [7]:
# ls_notResponse => TSV
length = 0
for ls in ls_notResponse:
    print(ls)
    length += 1
    
print(length)    
data1 = ls_notResponse
df = pd.DataFrame.from_dict(data1)
df.to_csv(r'C:\Users\joasi\geneBase\1_pub\Wyniki\ls_notResponse_200323.tsv', sep="\t")

6620401C13Rik
9330132O05Rik
9530053J19Rik
Gpr81
A830037N18Rik
9430023L20Rik
Cyr61
Gm6484
CKLF5
9830132E05Rik
Niacr1
LOC332300
LOC381739
LOC100040353
LOC672274
6030408B16Rik
LOC628050
3110023G01Rik
Nrp
Ccrn4l
1200012P04Rik
A530020H22Rik
9930013L23Rik
Ctgf
EG245297
D030063E12
9330177P18Rik
LOC624610
Fam129b
1700081H05Rik
4930544G21Rik
LOC100047902
Mllt4
Lrrc16
Grasp
LOC665946
Hmga2-ps1
BC020108
4632428N05Rik
Egfbp2
4930502E18Rik
D14Ertd668e
A530026G17
Mela
BC030863
LOC100041932
A130038J17Rik
1810011O10Rik
8430408G22Rik
Amica1
EG668311
EG640530
52


In [9]:
from biomart import BiomartServer
server = BiomartServer('http://www.ensembl.org/biomart')
dataset = server.datasets['mmusculus_gene_ensembl']

attributes = ['ensembl_gene_id','mgi_symbol']
filters = {'mgi_id':['MGI:95292',
                     'MGI:95537',
                     'MGI:106023',
                     'MGI:107565',
                     'MGI:109382',
                     'MGI:88613',
                     'MGI:1277133',
                     'MGI:1314653',
                     'MGI:1858441',
                     'MGI:1860303',
                     'MGI:1914701',
                     'MGI:1915368',
                     'MGI:1915982',
                     'MGI:1916318',
                     'MGI:1918730',
                     'MGI:1921298',
                     'MGI:1921765',
                     'MGI:1922263',
                     'MGI:1924234',
                     'MGI:1924802',
                     'MGI:1924879',
                     'MGI:1924967',
                     'MGI:1933383',
                     'MGI:2441671',
                     'MGI:2442377',
                     'MGI:2442733',
                     'MGI:2442910',
                     'MGI:2443629',
                     'MGI:2444394',
                     'MGI:2670959',
                     'MGI:2685484',
                     'MGI:3643534',
                     'MGI:3645360',
                     'MGI:3646928',
                     'MGI:3648717',
                     ]}

response = dataset.search({'attributes':attributes,'filters':filters})
print(response.text)


ENSMUSG00000023087	Noct
ENSMUSG00000068245	Phf11d
ENSMUSG00000068036	Afdn
ENSMUSG00000000531	Tamalin
ENSMUSG00000041957	Pkp2
ENSMUSG00000037204	Atg101
ENSMUSG00000021338	Carmil1
ENSMUSG00000056313	Tcim
ENSMUSG00000048489	Depp1
ENSMUSG00000020101	Vsir
ENSMUSG00000006191	Cdkal1
ENSMUSG00000055555	Ct55
ENSMUSG00000025420	Katnal2
ENSMUSG00000041889	Shisa4
ENSMUSG00000026163	Sphkap
ENSMUSG00000075408	Smim41
ENSMUSG00000045502	Hcar2
ENSMUSG00000049241	Hcar1
ENSMUSG00000042590	Ipo11
ENSMUSG00000051586	Mical3
ENSMUSG00000026796	Niban2
ENSMUSG00000052353	Cemip
ENSMUSG00000036412	Arsi
ENSMUSG00000048534	Jaml
ENSMUSG00000047822	Angptl8
ENSMUSG00000108022	Gm7298
ENSMUSG00000028195	Ccn1
ENSMUSG00000019997	Ccn2



In [8]:
# WYNIK_Excell
df.to_excel(r'C:\Users\joasi\geneBase\1_pub\Wyniki\notResponse_1_pub_2003233.xlsx', index=False)

In [ ]:
MGI_EntrezGene.rpt